In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.stats import linregress

In [2]:
def mbar_to_torr(mbar):
    return mbar*1e-3*1e5/(101325/760)
def torr_to_mbar(torr):
    return torr*1e3*1e-5*(101325/760)

In [3]:
vol_zyl=lambda r,l:np.pi*r**2*l
def zyl(obj):
    l,lerr,d,derr=obj
    r=d/2
    rerr=derr/2
    return np.pi*r**2*l,np.pi*2*(rerr*l+lerr*r)
rez1=(6.1,0.1,1.06,0.1)
ion=(0.85,0.05,0.58,0.05)
pen1=(0.75,0.05,0.7,0.05)
pen2=(0.6,0.05,0.25,0.05)
bar12=(0.92,0.05,0.73,0.05)
pir=(0.6,0.06,0.03,0.05)
tube=(10,1,0.2,0.05)
group=np.array([rez1,ion,pen1,pen2,bar12,bar12,pir,tube])
voloume=np.sum([zyl(obj)[0] for obj in group])
voloume_err=np.sum([zyl(obj)[1] for obj in group])
print(f"Das Voloumen beträgt {voloume:.1f}l mit einerm Fehler von {voloume_err:.1f}l")

Das Voloumen beträgt 7.0l mit einerm Fehler von 5.7l


In [4]:
file="test"
data=np.loadtxt(file)
time=[]
pressure=[]
for line in data:
    time.append(line[0])
    pressure.append(line[1])
time=np.array(time)
pressure=np.array(pressure)

In [7]:
linreg=linregress(time,pressure)
reg=lambda x: linreg.slope*x+linreg.intercept
reg_error=lambda x: linreg.stderr*x+linreg.intercept_stderr
print(f"Die Leckrate beträgt {torr_to_mbar(linreg.slope)*voloume} +- {linreg.stderr}")
print(linreg)

Die Leckrate beträgt 0.9247685752579695 +- 0.0018176758727099652
LinregressResult(slope=0.09894285714285714, intercept=0.028095238095238173, rvalue=0.999325697849158, pvalue=6.818717889510245e-07, stderr=0.0018176758727099652, intercept_stderr=0.05503286999644793)


In [ ]:
fig = plt.figure(figsize=(11, 6))
gs = GridSpec(8, 5)
fig1 = fig.add_subplot(gs[:, :])
fig1.set_title("Leckratenbestimmung")
fig1.set_ylabel("p in torr")
fig1.set_xlabel("t in s")
fig1.scatter(time,pressure,c="C0",label="Gemessener Druck")
fig1.plot(time,reg(time),c="C0",label="Linearer Fit")
fig1.plot(time,reg(time)+reg_error(time),c="C0",ls="--",label="Fehlerschlauch zum Fit")
fig1.plot(time,reg(time)-reg_error(time),c="C0",ls="--")
plt.tight_layout()
plt.legend()
plt.show()